In [1]:
import pandas as pd
import methods_collection.make_question as question_maker
import methods_collection.ranking_help_methods as help_methods
import pickle
import difflib
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [2]:
dataset = pd.read_csv('varieties_corpus/processed_word5perline_corpus.csv')
dataset = dataset.iloc[:,1:]
dataset = dataset.rename(columns={"Text": "Data"})
dataset.PageID = dataset.PageID.apply(lambda x: x.replace("Page_",""))
dataset.Data = dataset.Data.apply(lambda x: help_methods.clean_text(str(x)))
dataset.head()

,PageID,Data
0,0,カスタマイザー タイトル 設定 カスタマイザー サブタイトル
1,0,設定 MobiControl 設定 順序 MobiControl
2,0,設定 順序 MobiControl 設定 順序
3,0,行い 端末 OS 対象 オンプレミス
4,0,MobiControl サーバ 必要 な 設定


In [3]:
#help_methods.build_voc(dataset)
with open('customized_model/vocabulary.pkl', 'rb') as f:
    word_ids = pickle.load(f)

#help_methods.make_vector(dataset, word_ids)    
with open('customized_model/vectormodel_perpage_v1.pkl', 'rb') as f:
    vector = pickle.load(f)
    
len(vector)   

78244

In [11]:
sample_size = 5
questions_samples = question_maker.question_dataframe_generator_1000(dataset, sample_size)
questions_samples.head()

,Question,PageID
0,選択 できる 認証 れ 入れられますか,13
1,切断 表 端末 Mobicontrol 何ができますか,239
2,端末 一斉 送る ん 何ですか,245
3,管理 変更 追加 適用 何ができますか,324
4,cd ディレクトリ サンプル cd 何ですか,95


In [12]:
for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    input_vector = list(set(str(query).split()))
    query_vector = help_methods.custom_vectorizer(input_vector, word_ids)
    break

選択 できる 認証 れ 入れられますか 13


In [137]:
elements = '管理 変更 追加 適用 何ができますか 端末 一斉 送る ん 何ですか'
word_limit = 5
elements = elements.split()
chunks = [' '.join(elements[i:i + word_limit]) for i in range(0, len(elements), word_limit)]
chunks

['管理 変更 追加 適用 何ができますか', '端末 一斉 送る ん 何ですか']

In [146]:
chunks += ['abc', 234]

chunks

['管理 変更 追加 適用 何ができますか',
 '端末 一斉 送る ん 何ですか',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 234,
 'abc',
 234]